In [1]:
#Automatic film scanning for Film Carriers
#Using Arduino Nano
#MIT License
#Copyright (c) 2020 Seckin Sinan Isik

import cv2, time
import numpy as np
import pyfirmata
from scipy.stats import rankdata

In [2]:
board = pyfirmata.Arduino('/dev/cu.usbserial-14230') #Fill in YOUR PORT ADDRESS
it = pyfirmata.util.Iterator(board)
it.start()
Shutter    = board.get_pin('d:12:o')
Shutter.write(1) #starts with a 0 (False) on Arduino Nano
motMS1Pin  = board.get_pin('d:11:o')
motMS2Pin  = board.get_pin('d:10:o')
motMS3Pin  = board.get_pin('d:9:o')
motStepPin = board.get_pin('d:8:o')
motDirPin  = board.get_pin('d:7:o')
buttonRev     = board.get_pin('d:4:i')
buttonFrw     = board.get_pin('d:5:i')
buttonRun     = board.get_pin('d:6:i')
buttonPicture = board.get_pin('d:3:i')
laserInt      = board.get_pin('a:5:i')

In [36]:
# User input  
MaxPicutureNumber     = 28
BackupPictureTake     = 1    # When no image border is located, 0 means it doesn't backup to take a picture, 1 for yes
waitingTime           = 0.9  # 1 seconds after shuter is relesed, increase if pictures are blury 
colorNegative         = 1    # 0 for Color positive, 1 for All Negative films
framelength           = 111  # 111 for landscape 35mm, increase if progression falls short

In [37]:
# Intrinsic values 
HowManyFramesPerImage = 1    # Not yet developed 
OverlapValue          = 0.5  # Not yet developed 
BorderExpectation     = 10   # initial move is (x-1)/x th of the frame, 10 is ideal
CorrectionSpeed       = framelength*16 # resolution of a 1/16 micro stepper per frame
CorrectionSpeedP      = 832  # Portrait length, 1/16
framelengthP          = 52   # Portrait length , 1/1
orientation           = 0    # 0 for landscape 1 for portrait 
Cropfactor            = 0.2  # max 1 min 0, ideal 0.2
threshold             = 0.95 # border intensity is found needs to be 95% higher from the rest. 

In [ ]:
print("lets go")
latch           = 0    # used for second border searching
start           = 0 
counter         = 1    # picutre counter
autoAuto        = 0
BadFrames       = 0
FrameMoves      = 1
while True:
    ###########################################
    ################ Functions ################
    def my_label(res):
        label=np.zeros(len(res))
        label[0]=1
        for i in range(len(res)-1):            
            if res[i]==res[i+1]-1:
                label[i+1]=label[i]
            else:
                label[i+1]=label[i]+1
        return label
    def my_takeApicuture(counter):
        #print("picture number:", counter)
        counter=counter+1
        Shutter.write(0)
        time.sleep(0.3) #0.045 is able to trigger the shutter on a xt2 
        Shutter.write(1)
        time.sleep(waitingTime)
        return counter
        
    def my_functionFAST(xx,yy,a,b,c,wait):
        motDirPin.write(xx)
        motMS1Pin.write(a)
        motMS2Pin.write(b)
        motMS3Pin.write(c)            
        for x in range(yy):
            motStepPin.write(1)
            time.sleep(wait)
            motStepPin.write(0)
            time.sleep(wait)

    def my_maxEdge(FrameProjection,position): 

        if orientation==0:
            FrameLimit=len(FrameProjection) #this is Landscape 
            EdgeValue=5
        else:
            FrameLimit=len(FrameProjection) #this is portrait
            EdgeValue=5

        if min(position) >= 0+EdgeValue:
            a=float(FrameProjection[min(position)])
            b=float(FrameProjection[min(position)-3] )
            left=abs(a-b)
        if max(position) <= FrameLimit-EdgeValue:
            a=float(FrameProjection[max(position)])
            b=float(FrameProjection[max(position)+3])
            right=abs(a-b)
        if min(position) <= EdgeValue or max(position) >= FrameLimit-EdgeValue:
            return -1,-1
        else:
            return left,right
        
    def my_captureData():
        video=cv2.VideoCapture(2) #this could be 0 or 2 depending on your available inputs, such as webcam
        time.sleep(0.9)
        check, frame=video.read() #capturing screen  
        height=len(frame[:,0])
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        FrameProjection=np.sum(frame,axis=orientation )
        #getting rid of the extra sides from the screen capture
        CropVertical=round(height*Cropfactor) #cut out the effect of sprocket holes
        b=max(np.argwhere(FrameProjection>0))-5
        a=np.argmax(FrameProjection>0)+5
        frame=frame[0+CropVertical:height-CropVertical,a:int(b)] #cutoff 50 may change for video card
        FrameProjection=FrameProjection[a:int(b)]
        if colorNegative==0:    
            frame=~frame
            FrameProjection=max(FrameProjection)-FrameProjection
        #creating candidate borders
        maskHor = (FrameProjection/max(FrameProjection))>threshold  
        #labeling candidate borders
        position= np.where(maskHor==1)[0]  
        items=my_label(position)
        #bringing data together
        a=np.array([( x, 
                     len(position[items==x]),
                     round(np.median(position[items==x])),
                     max(np.sum(frame,axis=orientation)[position[items==x]])            
                    ) 
            for i, x in enumerate(np.unique(items))]) 
        # my_maxEdge(FrameProjection,position[items==x])
        PFramelength,LFramelength=np.shape(frame)
        return a,FrameProjection,position,items, PFramelength,LFramelength
    
    def my_CheckingEdgeDeciding(aa,FrameProjection,position,items,latch,BackupPictureTake):
        if orientation==0:
            a=11
            idealBorderSize=22 # typical for 35mm
        else:
            a=22
            idealBorderSize=44 # typical for 35mm
            
        aa[:,1]=abs(aa[:,1]-idealBorderSize)
        Positionselected=0
        weights=[[1],[4],[1]]
        if   latch ==  0:  #FORWARD expected border
            aa[:,2]=abs(aa[:,2]-round(LFramelength/BorderExpectation))
        elif latch == 2:  #BACKWARD expected border
            aa[:,2]=abs(aa[:,2]-(BorderExpectation-1.5)/BorderExpectation*LFramelength)
            
        decide= sum ( [rankdata(-aa[:,1]),rankdata(-aa[:,2]),rankdata(aa[:,3])]*np.array(weights) ) #applying decision weights 
        Itemselected=aa[decide==max(decide),0]                                #selected pulse as border
          
        
        # if the Selected border has a width >1, conatains informationm and there is one selected border
        if len(Itemselected)==1 and len(position[items==Itemselected])!=1:
            left,right=my_maxEdge(FrameProjection,position[items==Itemselected])
            if left>5*right and len(Itemselected)==1 and ( right>300 or left>300 ):     #prominant edge on the left
                #print("Prominent-L")
                Positionselected=min(position[items==Itemselected])+a
                if latch==0:
                    latch=0
                elif latch==2:
                    latch=1
            elif right>5*left and len(Itemselected)==1 and ( right>300 or left>300 ):  #prominant edge on the right
                #print("Prominent-R")
                Positionselected=max(position[items==Itemselected])-a
                if latch==0:
                    latch=0
                elif latch==2:
                    latch=1
            elif right>500 and left>500:                                         #there is some edge on both side
                #print("okEdge")
                Positionselected=round(np.median(position[items==Itemselected]))   
                if latch==0:
                    latch=0
                elif latch==2:
                    latch=1
            elif right==-1 and left==-1:
                if latch==0:
                    #print("FrameEdge not clear")
                    latch=2          #repeat the postion finding looop
                elif BackupPictureTake==1 and latch==2:
                    latch=3
                elif BackupPictureTake==0 and latch==2:
                    latch=4  
            else:
                #print("Poor PulseEdge")
                
                if latch==0:
                    latch=2          #repeat the postion finding looop
                elif BackupPictureTake==1 and latch==2:
                    latch=3      
                elif BackupPictureTake==0 and latch==2:
                    latch=4  
        else:
            #print("Poor PulseEdge")
            if latch==0:
                latch=2          #repeat the postion finding looop
            elif BackupPictureTake==1 and latch==2:
                latch=3
            elif BackupPictureTake==0 and latch==2:
                latch=4  
        return latch,Positionselected        
                      
    ###########################################
    ################## Main LOOP ############### 
    FullAuto                     = board.digital[6].read()
    FrameByFrameWithPicture      = board.digital[5].read()
    FrameByFrame                 = board.digital[4].read()
    ManualshutterButton          = board.digital[3].read()
    
    #latch 1 correction made, moving forward
    #latch 2 correction made on second border, moving backwards
    #latch 3 correction Not made, moving backwards and taking a picture (usually what you want, just incase)
    #latch 4 correction Not made, moving forwards looking for a new frame (made a bit too ambitious)
    
    if start==1 or autoAuto==3 or latch>=1:
        skip=0 
        # 1-Start by taking a picture
        if FrameMoves==1 and autoAuto>1 and HowManyFramesPerImage==1:
            counter=my_takeApicuture(counter)
        # 2-Initial film advancement
        if latch==0 and HowManyFramesPerImage==1: 
            my_functionFAST(1,round((BorderExpectation-1)/BorderExpectation*framelength),0,0,0,0.0008)
            time.sleep(0.1)
            FrameMoves=FrameMoves+1
        elif latch==2 and HowManyFramesPerImage==1:
            my_functionFAST(1,round((1+1.5)/BorderExpectation*framelength),0,0,0,0.0008)
        elif latch==0 and HowManyFramesPerImage>1 and FrameMoves>1:
            my_functionFAST(1,round(1/2*(framelength-3)),1,0,0,0.0008)
            
            
        # 2b-Multiple images per exposure 
        if HowManyFramesPerImage>1:
            latch=1
            FrameMoves=FrameMoves+1
            for i in range(HowManyFramesPerImage-1):                
                if autoAuto>1:
                    counter=my_takeApicuture(counter)
                my_functionFAST(1,round(framelength/(HowManyFramesPerImage+OverlapValue)),0,0,0,0.0008)  
        
        # 3-Capture FRAME to evaluate     
        a,FrameProjection,position,items, PFramelength,LFramelength = my_captureData() 
        # 4-Evaluating preliminary data and checking the features of the selected data
        latch,Positionselected=my_CheckingEdgeDeciding(a,FrameProjection,position,items,latch,BackupPictureTake)
 
        # 5-Borders were not found, but the user wants to scan the entire film strip
        if latch >=3 and HowManyFramesPerImage==1:   
            if latch==3:
                my_functionFAST(0,round((2.5-1)/BorderExpectation*framelength),0,0,0,0.0008)
                if autoAuto>1:
                    counter=my_takeApicuture(counter)
                    BadFrames=BadFrames+1
            elif latch==4:
                my_functionFAST(0,round((2.5-1)/BorderExpectation*framelength),0,0,0,0.0008)
            latch=0
            skip=1
        # 6-STOP if Picture Number>x and there is no information
        print(FrameMoves)
        if MaxPicutureNumber<FrameMoves: #do not excede maximum picture numper
            print("scan complete")
            latch=0
            autoAuto=0
            start=0
            
             
        # 7-Correction PART and taking picture
        if skip==0:
        
            if latch==0 and orientation==0:
        
                my_functionFAST(1,round(Positionselected/LFramelength*CorrectionSpeed),1,1,1,0.00008)
                if autoAuto>=2:
                    counter=my_takeApicuture(counter)
                    
            elif latch==1 and orientation==0:
             
                my_functionFAST(0,round( (LFramelength-Positionselected)/LFramelength*CorrectionSpeed),1,1,1,0.00008)
                latch=0
                if autoAuto>=2:
                    counter=my_takeApicuture(counter)
            elif latch==1 and orientation==1:
                
                my_functionFAST(0,round( (Positionselected) /PFramelength*CorrectionSpeedP),1,1,1,0.00008)
                latch=0
                if autoAuto>=2:
                    counter=my_takeApicuture(counter)
                
        else:
            skip=0
        # 8-Stop FrameByFrameWithPicture and FrameByFrame
        start=0   
            
    if  FullAuto     is False:
        autoAuto=3 #unlimited access to the function above 
        time.sleep(0.2)
    if FrameByFrameWithPicture is False:
        autoAuto=2 #limited access to the function above 
        start=1
        time.sleep(0.2)
    if FrameByFrame is False:
        autoAuto=1 #limited access to the function above 
        start=1
        time.sleep(0.2)
    if ManualshutterButton is False:  
        counter=my_takeApicuture(counter)


lets go
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
26
27
27
28
28
29
scan complete
